In [67]:
import pandas as pd

In [68]:
def is_numeric(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

In [89]:
results = pd.read_csv('./results/final.csv')
results['generated_text'] = [text.split('\n')[0].strip() for text in results['generated_text']]
results[['num', 'unit']] = results['generated_text'].str.split(n=1, expand=True)

In [109]:
range_indices = results['generated_text'].str.contains('\[| to ', regex=True)
ranges_results = results[range_indices]
normal_results = results[~range_indices]

<>:1: SyntaxWarning: invalid escape sequence '\['
<>:1: SyntaxWarning: invalid escape sequence '\['
/tmp/ipykernel_453406/2892454758.py:1: SyntaxWarning: invalid escape sequence '\['
  range_indices = results['generated_text'].str.contains('\[| to ', regex=True)


In [112]:
ranges_results[['num', 'unit']] = ranges_results['generated_text'].str.rsplit(n=1, expand=True)
ranges_results['num'] = ranges_results['num'].str.replace(']', '')
ranges_results[['_', 'num']] = ranges_results['num'].str.rsplit(n=1, expand=True)
ranges_results = ranges_results.drop(columns=['_'])
ranges_results

/tmp/ipykernel_453406/850656984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranges_results[['num', 'unit']] = ranges_results['generated_text'].str.rsplit(n=1, expand=True)
/tmp/ipykernel_453406/850656984.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranges_results['num'] = ranges_results['num'].str.replace(']', '')
/tmp/ipykernel_453406/850656984.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

,id,entity_name,generated_text,num,unit
19,19,voltage,"[220.0, 240.0] volt",240.0,volt
645,645,voltage,"[110.0, 140.0] volt",140.0,volt
1464,1464,voltage,"[18.0, 265.0] volt",265.0,volt
1873,1873,maximum_weight_recommendation,25 pound to 30 pound,30,pound
2639,2639,voltage,"[200.0, 240.0] volt",240.0,volt
...,...,...,...,...,...
128111,128111,maximum_weight_recommendation,"[8.0, 35.0] pound",35.0,pound
128177,128177,voltage,"[0.0, 20.0] volt",20.0,volt
129656,129656,item_weight,30 pound to 70 pound,70,pound
130209,130209,voltage,"[110.0, 120.0] volt",120.0,volt


In [116]:
def unit_checker(row):
    return row['unit'] in entity_unit_map[row['entity_name']]

In [115]:
invalid_indices = ~(ranges_results['num'].apply(is_numeric) & ranges_results['unit'].isin(allowed_units))
ranges_results[invalid_indices]

,id,entity_name,generated_text,num,unit


In [127]:
ranges_results.loc[~ranges_results.apply(unit_checker, axis=1), ['num', 'unit']] = ""

In [130]:
invalid_indices = ~(normal_results['num'].apply(is_numeric) & normal_results['unit'].isin(allowed_units))
normal_results.loc[invalid_indices, ['num', 'unit']] = ""

In [131]:
normal_results.loc[~normal_results.apply(unit_checker, axis=1), ['num', 'unit']] = ""

In [132]:
normal_results

,id,entity_name,generated_text,num,unit
0,0,height,1933.0 centimetre,1933.0,centimetre
1,1,width,1.0 inch,1.0,inch
2,2,height,20.0 centimetre,20.0,centimetre
3,3,depth,20.0 centimetre,20.0,centimetre
4,4,depth,10.0 inch,10.0,inch
...,...,...,...,...,...
131182,131182,maximum_weight_recommendation,1500 pound,1500,pound
131183,131183,item_weight,2.0 gram,2.0,gram
131184,131184,maximum_weight_recommendation,12.0 ton,12.0,ton
131185,131185,item_weight,123.0 pound,123.0,pound


In [133]:
ranges_results

,id,entity_name,generated_text,num,unit
19,19,voltage,"[220.0, 240.0] volt",240.0,volt
645,645,voltage,"[110.0, 140.0] volt",140.0,volt
1464,1464,voltage,"[18.0, 265.0] volt",265.0,volt
1873,1873,maximum_weight_recommendation,25 pound to 30 pound,30,pound
2639,2639,voltage,"[200.0, 240.0] volt",240.0,volt
...,...,...,...,...,...
128111,128111,maximum_weight_recommendation,"[8.0, 35.0] pound",35.0,pound
128177,128177,voltage,"[0.0, 20.0] volt",20.0,volt
129656,129656,item_weight,30 pound to 70 pound,70,pound
130209,130209,voltage,"[110.0, 120.0] volt",120.0,volt


In [158]:
combined_results = pd.concat([normal_results, ranges_results], axis=0)

In [160]:
combined_results 

,id,entity_name,generated_text,num,unit
0,0,height,1933.0 centimetre,1933.0,centimetre
1,1,width,1.0 inch,1.0,inch
2,2,height,20.0 centimetre,20.0,centimetre
3,3,depth,20.0 centimetre,20.0,centimetre
4,4,depth,10.0 inch,10.0,inch
...,...,...,...,...,...
128111,128111,maximum_weight_recommendation,"[8.0, 35.0] pound",35.0,pound
128177,128177,voltage,"[0.0, 20.0] volt",20.0,volt
129656,129656,item_weight,30 pound to 70 pound,70,pound
130209,130209,voltage,"[110.0, 120.0] volt",120.0,volt


In [161]:
combined_results = combined_results.sort_values('id').set_index('id')

In [162]:
combined_results['final_text'] = combined_results['num'] + ' ' + combined_results['unit']

In [163]:
combined_results

,entity_name,generated_text,num,unit,final_text
id,,,,,
0,height,1933.0 centimetre,1933.0,centimetre,1933.0 centimetre
1,width,1.0 inch,1.0,inch,1.0 inch
2,height,20.0 centimetre,20.0,centimetre,20.0 centimetre
3,depth,20.0 centimetre,20.0,centimetre,20.0 centimetre
4,depth,10.0 inch,10.0,inch,10.0 inch
...,...,...,...,...,...
131182,maximum_weight_recommendation,1500 pound,1500,pound,1500 pound
131183,item_weight,2.0 gram,2.0,gram,2.0 gram
131184,maximum_weight_recommendation,12.0 ton,12.0,ton,12.0 ton


In [153]:
test_df = pd.read_csv('./dataset/test.csv')

In [154]:
test_df

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth
...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight


In [164]:
test_df['prediction'] = combined_results['final_text']

In [165]:
test_df

,index,image_link,group_id,entity_name,prediction
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,1933.0 centimetre
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,1.0 inch
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,20.0 centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,20.0 centimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,10.0 inch
...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,1500 pound
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,2.0 gram
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,12.0 ton
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,123.0 pound


In [166]:
output = test_df[['index', 'prediction']]

In [167]:
output

,index,prediction
0,0,1933.0 centimetre
1,1,1.0 inch
2,2,20.0 centimetre
3,3,20.0 centimetre
4,4,10.0 inch
...,...,...
131182,131283,1500 pound
131183,131284,2.0 gram
131184,131285,12.0 ton
131185,131286,123.0 pound


In [169]:
output.to_csv('./results/test_out.csv')